# Preliminary ML fit

If unfamiliar with tensorflow, I suggest reading the [docs](https://www.tensorflow.org/guide/eager) before diving into this notebook.  However, I will also explain all the tf calls. The following todo is what this notebook is still missing, but I am releasing it now in the interest of timeliness. My next notebook will contain an exploration of other optimization algorithms, both with gradient and without.

Todo:
1. Estimate variance with replica
2. Replace explicit CFFs in network with preceeding dense layer as example of potential global fit

## Imports

In [1]:
%load_ext autoreload # whenever changes are made to any imported files this will reload them automatically
%autoreload 2

In [118]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [12]:
from BHDVCStf import BHDVCS #modified bhdvcs file
bhdvcs = BHDVCS()

In [4]:
df = pd.read_csv('dvcs_xs_newsets_genCFFs.csv')

## Utility class/func defs

This is a nice wrapper over my data to ensure that it always has the parameters in the correct order. It makes things so much easier.  Please feel free to extend and change it, but please post it if you do.

In [51]:
class DvcsData(object):
    def __init__(self, df):
        self.X = df.loc[:, ['phi_x', 'k', 'QQ', 'x_b', 't', 'F1', 'F2', 'ReH', 'ReE', 'ReHtilde', 'dvcs']]
        self.XnoCFF = df.loc[:, ['phi_x', 'k', 'QQ', 'x_b', 't', 'F1', 'F2', 'dvcs']]
        self.y = df.loc[:, 'F']
        
    def __len__(self):
        return len(self.X)
    
    def getSet(self, setNum, itemsInSet=36):
        pd.options.mode.chained_assignment = None
        subX = self.X.loc[setNum*itemsInSet:(setNum+1)*itemsInSet-1, :]
        subX['F'] = self.y.loc[setNum*itemsInSet:(setNum+1)*itemsInSet-1]
        pd.options.mode.chained_assignment = 'warn'
        return DvcsData(subX)
        

This is a vectorized wrapper of Liliet's TotalUUXS function

In [7]:
def vecF(DvcsData, TotalUUXS):
    """
    params:
        data: this should be of type DvcsData
        TotalUUXS: this should be the function from F.C
    """
    results = np.zeros(len(data))
    for i in range(len(data)):
        results[i] = TotalUUXS(*data.X.loc[i, :])
    return results

A data container for the loss and cff values at each epoch.

In [98]:
class savedParams(object):
    def __init__(self, numEpochs):
        self.savedparams = pd.DataFrame({'Epoch':np.zeros(numEpochs), 'Loss':np.zeros(numEpochs),
                                         'ReH':np.zeros(numEpochs), 'ReE':np.zeros(numEpochs),
                                         'ReHtilde':np.zeros(numEpochs)})
    
    def newData(self, epoch, loss, ReH, ReE, ReHtilde):
        self.savedparams.loc[epoch, :] = {'Epoch':epoch, 'Loss':loss, 'ReH':ReH, 'ReE':ReE, 'ReHtilde':ReHtilde}

In [104]:
def pcterr(obs, exp):
    return 100*(obs-exp)/exp

Utility function to format printouts at each epoch.

In [97]:
def form(tensors): #only works for 1d tensors
    return str(np.round(np.array([x.numpy() for x in tensors]), decimals=2))

# Check BHDVCStf for accuracy

Whenever modifications are made to the BHDVCS python file it should be checked against Liliet's known correct file.

In [8]:
from ROOT import gROOT

Welcome to JupyROOT 6.22/00


In [9]:
gROOT.LoadMacro('F.C')

0

In [10]:
from ROOT import F # not vectorized can only receive scalars
f = F()

In [24]:
data = DvcsData(df)

In [33]:
resLiliets = vecF(data, f.TotalUUXS)
resTF = bhdvcs.TotalUUXS_curve_fit(np.asarray(data.XnoCFF).T, data.X['ReH'], data.X['ReE'], data.X['ReHtilde'])

In [37]:
all(resTF.numpy() == resLiliets)

True

All the values are identical so we can conclude that there are no errors in BHDVCStf.py

# Fit TotalUUXS With Adam Optimizer

This is functionally identical to the $\chi^2$ minimization in scipy or root, except with Adam instead of levenberg-marquardt or trust-region.  However, it is useful as a demonstration of the TotalUUXS function as a layer in the
tensorflow graph.  It is run in eager execution for ease in experimentation.

In this example we are just using the first set of data.

In [52]:
data = DvcsData(df)
set0 = data.getSet(0)
X_train = np.asarray(set0.XnoCFF).T # have to take transpose to get everything to work
y_train = np.asarray(set0.y)

The class below defines the TotalUUXS layer.  In tensorflow, parameters are decalared as a tf.Variable, so the three compton form factors are set thus.

In [43]:
class TotalUUXS(tf.keras.Model):
    def __init__(self):
        super(TotalUUXS, self).__init__(dtype='float64')
        self.ReH = tf.Variable(1., dtype='float64', name='ReH') # all compton form factors are set to 1.0 initially
        self.ReE = tf.Variable(1., dtype='float64', name='ReE')
        self.ReHtilde = tf.Variable(1., dtype='float64', name='ReHtilde')
        self.F = BHDVCS()
    def call(self, inputs):
        return self.F.TotalUUXS(inputs, self.ReH, self.ReE, self.ReHtilde)

The loss is the mean squared error ($\chi^2$) between the output of the totalUUXS and the F in the dataset.

In [49]:
def loss(model, inputs, targets):
    error = model(inputs) - targets
    return tf.reduce_mean(tf.square(error))

If everything is calculated inside a GradientTape, the gradient of each tf.variable w.r.t. the loss can be calculated.  This function returns the gradients of ReH, ReE, and ReHtilde w.r.t. the loss function (i.e. how much the loss changes as each of the compton form factors change).

In [39]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return tape.gradient(loss_value, [model.ReH, model.ReE, model.ReHtilde])

This is my training loop.  As a general matter, it looks like the picture below. In this case the model is TotalUUXS, the loss is the one we defined, the gradient step is the the apply_gradients line in the cell below, and "step" is the next iteration of the loop.  I haven't used any callbacks beyond saving the best compton form factors, and that's implemented directly instead of as a callback.  If your loop gets more complicated, though, one would perhaps benefit from using (tensorflow's training loop and callbacks)[https://www.tensorflow.org/guide/keras/train_and_evaluate].

![](https://dzlab.github.io/assets/2019/20190316-training_loop.png)

In [116]:
def training_loop(epochs, X_train, y_train, lr=5000, when2print=None):
    
    sv = savedParams(epochs)
    model = TotalUUXS()  # Should maybe be refactored
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr) # in this case we need a pretty high learning rate
    
    for i in range(epochs):
        grads = grad(model, X_train, y_train)
        optimizer.apply_gradients(zip(grads, [model.ReH, model.ReE, model.ReHtilde]), )

        epoch_loss = loss(model, X_train, y_train)
        sv.newData(i, epoch_loss.numpy(), model.ReH.numpy(), model.ReE.numpy(), model.ReHtilde.numpy())
        if when2print:
            if i % when2print == 0: # print state every __ epochs
                print("Loss at epoch {:03d}: {:.5f}".format(i, epoch_loss),
                      "Grads: " + form(grads),
                      "ReH, ReE, ReHtilde: " + form([model.ReH, model.ReE, model.ReHtilde]))
    return sv

In [100]:
info = training_loop(3000, X_train, y_train when2print=50)

Loss at epoch 000: 16447.75923 Grads: [-0. -0.  0.] ReH, ReE, ReHtilde: [ 4985.45  4921.98 -4669.43]
Loss at epoch 050: 3.56026 Grads: [-0.28 -0.05  0.01] ReH, ReE, ReHtilde: [ -60.39 -122.75   83.87]
Loss at epoch 100: 0.09907 Grads: [ 0.03  0.01 -0.  ] ReH, ReE, ReHtilde: [ 24.64 -37.82  -4.63]
Loss at epoch 150: 0.00242 Grads: [-0. -0.  0.] ReH, ReE, ReHtilde: [ 10.57 -52.04   8.97]
Loss at epoch 200: 0.00001 Grads: [ 0.  0. -0.] ReH, ReE, ReHtilde: [ 12.41 -50.38   7.34]
Loss at epoch 250: 0.00001 Grads: [-0. -0.  0.] ReH, ReE, ReHtilde: [ 12.54 -50.45   7.22]
Loss at epoch 300: 0.00001 Grads: [-0. -0.  0.] ReH, ReE, ReHtilde: [ 12.57 -50.64   7.22]
Loss at epoch 350: 0.00001 Grads: [-0.  0. -0.] ReH, ReE, ReHtilde: [ 12.61 -50.83   7.22]
Loss at epoch 400: 0.00001 Grads: [-0.  0.  0.] ReH, ReE, ReHtilde: [ 12.65 -51.04   7.22]
Loss at epoch 450: 0.00001 Grads: [-0.  0.  0.] ReH, ReE, ReHtilde: [ 12.7  -51.26   7.21]
Loss at epoch 500: 0.00001 Grads: [-0.  0.  0.] ReH, ReE, ReHtild

### PctErr of ReH, ReE, ReHtilde

In [102]:
minloss = info.savedparams.loc[info.savedparams['Loss'].idxmin(), :]

In [109]:
for cff in ['ReH', 'ReE', 'ReHtilde']:
    print(cff + ": " + str(pcterr(minloss[cff], df.loc[0, cff])))

ReH: 6.623361917190539
ReE: 8.475008187218595
ReHtilde: -1.868313218395556


# Fits for all Kinematic Sets

In [111]:
data = DvcsData(df) # I am repeating this just to make it clear

In [132]:
numsets = df["#Set"].max()+1

pcterrs = pd.DataFrame({
  "ReH": np.zeros(numsets),
  "ReE": np.zeros(numsets),
  "ReHtilde": np.zeros(numsets)
})

In [133]:
for i in tqdm(range(numsets)):
    setI = data.getSet(i)
    X_train = np.asarray(setI.XnoCFF).T # have to take transpose to get everything to work
    y_train = np.asarray(setI.y)
    info = training_loop(5000, X_train, y_train)
    minloss = info.savedparams.loc[info.savedparams['Loss'].idxmin(), :]
    
    for cff in ['ReH', 'ReE', 'ReHtilde']:
        pcterrs.loc[i, cff] = pcterr(minloss[cff], df.loc[0, cff])


100%|██████████| 15/15 [42:41<00:00, 170.78s/it]


In [135]:
pcterrs

,ReH,ReE,ReHtilde
0,6.841211,8.750153,-1.899631
1,-23.032953,-26.754834,17.403152
2,-50.335061,-19.204528,-36.724805
3,10.185883,54.322118,-10.689371
4,-8.235227,-6.967373,5.328316
5,-48.504747,-16.507414,-39.898916
6,-24.065140,-22.077185,2.563795
7,0.319300,41.979024,10.807291
8,-5.460770,-3.362521,5.718416
9,-38.096585,0.895393,-54.397170


## Average percent error for each CFF

In [136]:
for cff in ['ReH', 'ReE', 'ReHtilde']:
    print(cff + ": " + str(np.mean(np.abs(pcterrs[cff]))))

ReH: 25.46964818873308
ReE: 16.932177576928304
ReHtilde: 22.23093123037195


## Discussion

These fits are much worse on average than the baseline fit NewsetExtraction.ipynb, which had average absolute standard errors of 9.8%, 10.5%, and 8%, and they took much longer to compute.  This may be fixable, as there are a number of hyperparameters in the Adam optimizer that can be adjusted.